In [1]:
import os
import json
import nltk
import math
import numpy as np
from nltk.tokenize import TweetTokenizer

In [2]:
tknzr = TweetTokenizer(reduce_len=True)

videoLength = 28662 # temp const
numberOfMessage = 1276 # temp const

In [3]:
def normalized_shannon_entropy(text): # entropy diversity measure (normalized)
    entropy = 0
    
    textLength = len(text)
    if textLength <= 1:
        return 0
    else:
        vocabulary = set(text)
        for word in vocabulary:
            p = text.count(word) / textLength

            entropy -= p * math.log2(p)

        return entropy / math.log2(textLength)

In [6]:
messages = [[] for i in range(videoLength)]
for i in range(1, numberOfMessage + 1):
    with open("../TwitchHighlightCrawler/vod/lirik/389178879/Message-" + str(i) + ".json", "r", encoding="utf-8") as file:
        data = file.read()
    commentData = json.loads(data)['comments']
    for comment in commentData:
        offset = math.floor( comment['content_offset_seconds'] ) # get comment offset

        if offset >= videoLength:
            break

        messages[offset].append( comment['message']['body'] )
        
i = 0
clip_distribution = np.zeros(101, np.int64)
while i < videoLength:
    localMessage = ''
    for j in range(i, i+30):
        if j >= videoLength:
            break
        
        localMessage += ' '.join(messages[j]) + ' '
            
    localTokens = tknzr.tokenize(localMessage) # tokenization
    localText = nltk.text.Text(localTokens) # convert tokens to NLTK text

    score = normalized_shannon_entropy(localText)
    clip_distribution[ math.floor(score * 100) ] += 1
    
    i += 30

In [7]:
print(clip_distribution)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  1  1  1  1  0  0  2  0  2  1  1  2  0  1  5  1  1  2
  1  2  0  5  2  1  3  3  0  1  1  2  1  0  1  2  1  0  4  2 10  5 11 15
  4 11  8 17 18 25 24 26 28 28 45 49 43 53 63 68 68 65 60 62 44 18 16 10
  4  2  1  0  0]
